<a href="https://colab.research.google.com/github/skywalker0803r/mmpose_api/blob/main/api%E4%BD%BF%E7%94%A8%E7%AF%84%E4%BE%8B_%E5%BD%B1%E7%89%87%E7%89%88api.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
%%time
import requests
import time

file_path = '/content/drive/MyDrive/Baseball Movies/CH_videos_4s/pitch_0001.mp4'
# 設定 API 的 URL
#可以ping 但不要太長ping 我怕免費額度爆掉
url = "https://mmpose-api-origin-924124779607.europe-west1.run.app/pose_video"  # 替換為你的 API URL
# 使用 with 開啟檔案並傳送
with open(file_path, 'rb') as video_file:
    # 以 multipart/form-data 格式上傳檔案
    files = {'file': (file_path, video_file, 'video/mp4')}
    s = time.time()
    response_origin = requests.post(url, files=files)
    e = time.time()
origin_time = e-s
print('原本推論時間:',origin_time)

url = "https://mmpose-api-fast-924124779607.europe-west1.run.app/pose_video"  # 替換為你的 API URL
# 使用 with 開啟檔案並傳送
with open(file_path, 'rb') as video_file:
    # 以 multipart/form-data 格式上傳檔案
    files = {'file': (file_path, video_file, 'video/mp4')}
    s = time.time()
    response_fast = requests.post(url, files=files)
    e = time.time()
fast_time = e-s
print('加速推論時間:',fast_time)

原本推論時間: 15.317528009414673
加速推論時間: 8.912722110748291
CPU times: user 250 ms, sys: 28.4 ms, total: 279 ms
Wall time: 24.2 s


In [8]:
import cv2
import numpy as np

COCO_CONNECTIONS = [
    (0, 1), (0, 2), (1, 3), (2, 4),
    (5, 6), (5, 7), (7, 9),
    (6, 8), (8, 10),
    (5, 11), (6, 12), (11, 12),
    (11, 13), (13, 15), (12, 14), (14, 16)
]

def draw_pose(frame, keypoints, scores, min_score=0.5):
    for i, (x, y) in enumerate(keypoints):
        if i < len(scores) and scores[i] > min_score:
            cv2.circle(frame, (int(x), int(y)), 4, (0, 255, 0), -1)

    for i, j in COCO_CONNECTIONS:
        if i < len(keypoints) and j < len(keypoints):
            if scores[i] > min_score and scores[j] > min_score:
                x1, y1 = keypoints[i]
                x2, y2 = keypoints[j]
                cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)

def render_video_with_pose_fast(video_path: str, response_json: dict, output_path: str = "output_pose.mp4", min_score_thresh=0.5) -> str:
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise IOError(f"Cannot open video: {video_path}")

    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*"mp4v")
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

    # 建立 frame_idx -> prediction 映射
    frame_map = {f["frame_idx"]: f.get("predictions", []) for f in response_json.get("frames", [])}

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # 多人姿態列表包在 predictions[0]
        people_list = frame_map.get(frame_idx, [])
        if people_list and isinstance(people_list[0], list):
            people_list = people_list[0]  # 解開雙層 list

        for person in people_list:
            kpts = person.get("keypoints", [])
            scores = person.get("keypoint_scores", [])
            if kpts and scores:
                draw_pose(frame, np.array(kpts), np.array(scores), min_score_thresh)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    return output_path


In [9]:
pose_result = response_origin.json()  # 或讀入 json 檔
video_path = file_path
output_path = render_video_with_pose_fast(video_path, pose_result, output_path="origin.mp4")
print(f"輸出影片：{output_path}")

pose_result = response_fast.json()  # 或讀入 json 檔
video_path = file_path
output_path = render_video_with_pose_fast(video_path, pose_result, output_path="fast.mp4")
print(f"輸出影片：{output_path}")


輸出影片：origin.mp4
輸出影片：fast.mp4
